In [46]:
import pandas as pd
import numpy as np
import math 

train = pd.read_csv('data/stores_train.csv', nrows= 2000)
def dist_to_all_km(lat, lon, df):

    # coordinates in radians
    lat1 = lat*math.pi/180
    lon1 = lon*math.pi/180
    lat2 = df['lat']*math.pi/180 # go through whole lat column
    lon2 = df['lon']*math.pi/180 # go through whole lon column

    # store original coordinates in new dataframe
    distances = pd.DataFrame()
    distances['lat'] = df['lat'].copy()
    distances['lon'] = df['lon'].copy()

    # calculate cartesian coordinates
    R = 6371 # Earth radius in km
    df['x'] = R*np.cos(lat2)*np.cos(lon2)
    df['y'] = R*np.cos(lat2)*np.sin(lon2)
    df['z'] = R*np.sin(lat2)
    x1 = R*np.cos(lat1)*np.cos(lon1)
    y1 = R*np.cos(lat1)*np.sin(lon1)
    z1 = R*np.sin(lat1)

    # calculate distance, store as new column in the distances dataframe
    distances['dist'] = np.sqrt(np.square(df['x']-x1)+np.square(df['y']-y1)+np.square(df['z']-z1))

    return distances['dist'].squeeze()


In [47]:
a = dist_to_all_km(59, 10, train)
print(dist_to_all_km(59, 10, train))
print(type(a))

0         83.440246
1        109.742678
2        130.302897
3        945.569713
4        484.892100
           ...     
1995     123.603020
1996     596.192053
1997    1197.496522
1998     117.259350
1999     493.150570
Name: dist, Length: 2000, dtype: float64
<class 'pandas.core.series.Series'>


In [48]:

train = pd.read_csv('data/stores_train.csv')
train = train.drop('chain_name', axis=1)
train = train.drop('mall_name', axis=1)
train = train.drop('sales_channel_name', axis=1)
train['lv2'] = train['plaace_hierarchy_id'].astype(str).str[:5]
train['closest_store'] = None
train['mean_revenue_of_similar_stores_within 2km'] = None
train['mean_revenue_of_similar_stores_within_5km'] = None
train['number_of_stores_2km'] = None
train['number_of_stores_5km'] = None
train['median_2km'] = 0
train['median_5km'] = 0

num = 0 
mylist = []
for index in range(len(train)):
    lat = train._get_value(index,'lat')
    lon = train._get_value(index,'lon')
    lvl = train._get_value(index, 'lv2')
    dist_to_all = dist_to_all_km(lat, lon, train)

    #closest competitor
    dist_to_all = dist_to_all.sort_values(ascending=True)
    closest_competitor = None
    distiter = 0
    for dist in dist_to_all:
        if train._get_value(distiter, 'lv2') == lvl and dist != 0:
            closest_competitor = dist
        distiter += 1

    num2k = 0
    rev2k = []
    num5k = 0
    rev5k = []

    iter = 0

    for distance in dist_to_all:
        if distance < 2 and train._get_value(iter, 'lv2') == lvl:
            rev2k.append(train._get_value(iter, 'revenue'))
            num2k += 1

        if distance < 5 and train._get_value(iter, 'lv2') == lvl:
            rev5k.append(train._get_value(iter, 'revenue'))
            num5k += 1
        iter += 1
    num += 1
    print(num)

    train._set_value(index, 'closest_store', float(closest_competitor))
    #train._set_value(index, 'mean_revenue_of_similar_stores_within 2km', sum(rev2k)/num2k)
    #train._set_value(index, 'mean_revenue_of_similar_stores_within_5km', sum(rev5k)/num5k)
    #train._set_value(index, 'number_of_stores_2km', num2k)
    #train._set_value(index, 'number_of_stores_5km', num5k)
    #
    #train._set_value(index, 'median_2km', float(np.median(rev2k)))
    #train._set_value(index, 'median_5km', float(np.median(rev5k)))


    num+=1 #for runtime tracking only

train.head(20)

1
3
5
7
9
11
13
15
17
19
21
23
25
27
29
31
33
35
37
39
41
43
45
47
49
51
53
55
57
59
61
63
65
67
69
71
73
75
77
79
81
83
85
87
89
91
93
95
97
99
101
103
105
107
109
111
113
115
117
119
121
123
125
127
129
131
133
135
137
139
141
143
145
147
149
151
153
155
157
159
161
163
165
167
169
171
173
175
177
179
181
183
185
187
189
191
193
195
197
199
201
203
205
207
209
211
213
215
217
219
221
223
225
227
229
231
233
235
237
239
241
243
245
247
249
251
253
255
257
259
261
263
265
267
269
271
273
275
277
279
281
283
285
287
289
291
293
295
297
299
301
303
305
307
309
311
313
315
317
319
321
323
325
327
329
331
333
335
337
339
341
343
345
347
349
351
353
355
357
359
361
363
365
367
369
371
373
375
377
379
381
383
385
387
389
391
393
395
397
399
401
403
405
407
409
411
413
415
417
419
421
423
425
427
429
431
433
435
437
439
441
443
445
447
449
451
453
455
457
459
461
463
465
467
469
471
473
475
477
479
481
483
485
487
489
491
493
495
497
499
501
503
505
507
509
511
513
515
517
519
521
523
525
527

,store_id,year,store_name,plaace_hierarchy_id,grunnkrets_id,address,lat,lon,revenue,lv2,closest_store,mean_revenue_of_similar_stores_within 2km,mean_revenue_of_similar_stores_within_5km,number_of_stores_2km,number_of_stores_5km,median_2km,median_5km,x,y,z
0,983540538-974187930-44774,2016,MCDONALD'S BRAGERNES TORG MAGASINET,1.1.1.0,6020303,BRAGERNES TORG 13,59.743104,10.204928,17.998,1.1.1,0.130255,None,None,None,None,0,0,3159.422103,568.750298,5503.109673
1,987074191-973117734-44755,2016,MCDONALD'S KLINGENBERGGATA,1.1.1.0,3010306,NaN,59.913759,10.734031,23.828,1.1.1,0.116439,None,None,None,None,0,0,3137.917127,594.846117,5512.646842
2,984890265-981157303-64491,2016,BURGER KING HØNEFOSS,1.1.1.0,6050102,KONG RINGS GATE 1,60.164751,10.254656,16.099,1.1.1,0.116864,None,None,None,None,0,0,3118.990969,564.267188,5526.584739
3,914057442-992924179-126912,2016,BURGER KING GLASSHUSPASSASJEN,1.1.1.0,18040102,STORGATA 12,67.283669,14.379796,9.296,1.1.1,0.064965,None,None,None,None,0,0,2383.203886,611.007066,5876.789141
4,913018583-913063538-668469,2016,BURGER KING TILLERTORGET,1.1.1.0,16017414,NaN,63.358068,10.374832,4.528,1.1.1,0.743934,None,None,None,None,0,0,2810.134063,514.480268,5694.567376
5,998035295-979540094-54657,2016,BURGER KING STAVANGER,1.1.1.0,11030509,TORGET 7,58.970249,5.731728,12.020,1.1.1,0.091616,None,None,None,None,0,0,3267.723419,327.989792,5459.308322
6,916757433-916772262-822439,2016,BURGER KING NARVIK,1.1.1.0,18050706,BOLAGSGATA 1,68.438025,17.422916,7.622,1.1.1,0.250731,None,None,None,None,0,0,2233.968273,701.064604,5925.161194
7,913158733-913203305-675470,2016,GRENSEN BURGERBAR,1.1.1.0,1010301,OLAV VS GATE 1,59.119175,11.389572,3.640,1.1.1,0.498875,None,None,None,None,0,0,3205.547103,645.744581,5467.826141
8,982174139-982239737-14482,2016,MCDONALD'S ÅSANE SENTER,1.1.1.0,12010915,ÅSANE SENTER 37,60.465118,5.324101,30.514,1.1.1,0.497131,None,None,None,None,0,0,3127.056413,291.414986,5543.125156
9,916217153-911834553-435311,2016,BERGBYS NOTODDEN,1.1.1.0,8070301,NESØYA,59.559079,9.249380,6.339,1.1.1,1.239542,None,None,None,None,0,0,3185.896569,518.820825,5492.770613


In [ ]:
def modify(data):
    sdsdsd
    return data
train_data = modify(train_data)